In [360]:
# coding: utf-8
"""
igraphを使ったグラフクラスタリング
greedyアルゴリズム（louvain法）によってクラスタリングした後
サブグラフの中心性指標を図ることで単語の順位付けを行う
媒介中心性と固有ベクトル中心性の２種類を採用
"""
from igraph import *
import csv
import collections

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    list_edge = [tuple(sorted(row)) for row in list_edge]
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    # エッジリストとそのweightを作成
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    return {"vertex": list_vertices, "edge": list(tuple_edge), "weight": list(tuple_weight)}

# クラスタリング済みのネットワークを元にサブグラフのリストを作成
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            if row[0] in cluster_vertex and row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": cluster_vertex,
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_vertex, cluster_edge, cluster_weight
                         in zip(list_cluster_vertex, list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

メイン部分

In [363]:
# エッジリストの読み込み
list_edge = readcsv("./files/text_for_cytoscape_rev.csv")
# 元のネットワークを作成する（無向）
dict_network_master = cal_master_network(list_edge)
# g = Graph(directed=True)
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
# louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
# クラスタ結果をもとにサブグラフのリストを作成
list_dict_network_sub = cal_cluster_to_network(dict_network_master)
# サブクラスタごとに中心性の計算
for i, dict_network_sub in enumerate(list_dict_network_sub):
    g_sub = Graph()
    g_sub.add_vertices(dict_network_sub["vertex"])
    g_sub.add_edges(dict_network_sub["edge"])
    list_dict_network_sub[i]["center_bet"] = g_sub.betweenness(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["center_eigen"] = g_sub.eigenvector_centrality(directed=False, weights=dict_network_sub["weight"])

プロット部分

In [372]:
# クラスタ番号 
num = 6

list_word_bet = sorted(zip(list_dict_network_sub[num]["vertex"], list_dict_network_sub[num]["center_bet"]), key=lambda x: x[1], reverse=True)
list_word_eigen = sorted(zip(list_dict_network_sub[num]["vertex"], list_dict_network_sub[num]["center_eigen"]), key=lambda x: x[1], reverse=True)

for word in zip(list_word_bet, list_word_eigen):
    print word[0][0], "|||||", word[1][0]

メニュー ||||| 栄養
料理 ||||| バランス
味 ||||| 管理
自分 ||||| メニュー
美味しい ||||| 料理
気 ||||| 健康
調理 ||||| 豊富
悪い ||||| 栄養士
提供 ||||| カロリー
用意 ||||| 自分
使用 ||||| 美味しい
管理 ||||| 提供
栄養士 ||||| 好き
体調 ||||| 管理栄養士
美しい ||||| 用意
豊富 ||||| おいしい
和食 ||||| 配慮
特別 ||||| 種類
ミキサー ||||| 味
四季 ||||| 状態
選択 ||||| 健康的
献立 ||||| 洋食
通常 ||||| 見た目
味付け ||||| 味付け
気分 ||||| 選択
食材 ||||| 不安
基本的 ||||| 特別
好み ||||| 好み
栄養 ||||| 食材
健康的 ||||| 和食
野菜 ||||| 野菜
来客 ||||| 体調
カロリー ||||| 中心
おいしい ||||| 調理
配慮 ||||| 使用
希望 ||||| 工夫
お願い ||||| 細やか
状態 ||||| 朝食
洋食 ||||| 手作り
急 ||||| 薄味
忙しい ||||| バリエーション
農園 ||||| 通常
要素 ||||| 悪い
専属 ||||| 献立
高齢 ||||| 気
種類 ||||| 記載
バランス ||||| 来客
冷蔵庫 ||||| 素材
健康 ||||| 夕食
こだわり ||||| 家具
春 ||||| ランチ
好き ||||| 専属
シェフ ||||| 変化
温かい ||||| 美しい
最初 ||||| 予約
お世話 ||||| 基本的
薄味 ||||| 基本
工夫 ||||| こだわり
事前 ||||| 定食
年寄り ||||| 四季
糖尿病 ||||| 冷たい
予約 ||||| 希望
変化 ||||| 事前
夕食 ||||| 要素
旬の食材 ||||| 1日
見た目 ||||| 個々
ランチ ||||| 母親
おやつ ||||| 試食
試食 ||||| 食
ヒーター ||||| 歯
不安 ||||| 計算
食器 ||||| 食器
品数 ||||| 気分
遊び ||||| 冷蔵庫
日替わり ||||| 旬の食材
定食 ||||| パン
誕生日 ||||| 幸せ
流動食 ||||| 料理長
